In [19]:
import os
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
from dotenv import load_dotenv
from openai import OpenAI

In [9]:
load_dotenv()

True

In [22]:
# 텍스트를 음성 파일로 저장하고, pydub으로 저장하는 함수
def speak(text):
    print(f"🤖: {text}")
    try:
        tts = gTTS(text=text, lang='ko')
        tts.save('quizbot_response.mp3')

        audio = AudioSegment.from_mp3('quizbot_response.mp3')
        play(audio)
    except Exception as e:
        print(f"음성 출력 중 에러 발생: {e}")
    finally:
        if os.path.exists('quizbot_response.mp3'):
            os.remove('quizbot_response.mp3')

In [23]:
# 사용자의 음성을 텍스트로 변환하는 함수
def listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎙️듣고 있어요. 말씀해주세요..")
        try:
            audio = r.listen(source, timeout=5, phrase_time_limit=10)
            text = r.recognize_google(audio, language='ko-KR')
            print(f"👤사용자: {text}")
            return text
        except sr.UnknownValueError:
            speak("죄송하지만, 잘 알아듣지 못했어요. 다시 말씀해 주시겠어요?")
            return ""
        except sr.WaitTimeoutError:
            return "TIMEOUT"

In [25]:
while True:
    # --- 1. 프로젝트 설정 및 변수 초기화 ---
    client = OpenAI()

    score = 0
    wrong_answers = []
    exit_game = False

    # --- 2. AI 퀴즈쇼 호스트 페르소나 정의 및 문제 생성 ---
    system_prompt = """
    당신은 '퀴즈 갈매기'라는 이름의 롯데 자이언츠 퀴즈쇼 호스트입니다.
    '롯데 자이언츠'에 대한 '초급 난이도'의 퀴즈 3개를 문제와 정답 형식으로 JSON 리스트로 만들어주세요.

    #--- 규칙 ---#
    1. 매우 중요: 출제하는 모든 퀴즈의 정답은 반드시 검증된 사실이어야 하며, 절대 지어내면 안 된다.
    2. 퀴즈는 구단 정보, 선수 정보 등 명확한 사실에 기반한 질문으로만 구성한다. 마스코트 질문은 배제한다.

    
    JSON 형식 예시:
    [
      {
        "question": "롯데 자이언츠의 홈 구장 이름은 무엇일까요?",
        "answer": "사직 야구장"
      },
      ...
    ]
    """

    print("AI가 새로운 퀴즈를 출제하는 중입니다...")
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": "퀴즈 3개를 JSON 형식으로 생성해줘."}
            ],
            temperature=0.7,
        )
        import json
        quiz_list = json.loads(response.choices[0].message.content) 
        
        # --- 3. 퀴즈쇼 시작 ---
        speak("안녕하세요, 저는 롯데 자이언츠 퀴즈쇼 호스트, 퀴즈 갈매기입니다! 지금부터 총 3문제를 풀어볼 거예요.")

        # --- 4. 퀴즈 반복문 ---
        for i, quiz in enumerate(quiz_list):
            speak(f"자, {i+1}번째 문제입니다! {quiz['question']}")
            
            user_answer = listen()

            # 종료 명령어 확인
            if "종료" in user_answer:
                speak("퀴즈를 종료합니다. 이용해주셔서 감사합니다.")
                exit_game = True
                break
            
            # 시간 초과 처리
            if user_answer == "TIMEOUT":
                speak(f"시간 초과! 아쉽지만 오답입니다. 정답은 '{quiz['answer']}'이었어요.")
                wrong_answers.append({
                    "question": quiz['question'],
                    "correct_answer": quiz['answer'],
                    "my_answer": "시간 초과"
                })

            # 정답 확인
            elif quiz['answer'].replace(" ", "") in user_answer.replace(" ", "") and user_answer != "":
                speak("딩동댕! 정답입니다!")
                score += 1

            # 오답 처리
            else:
                speak(f"아~ 아쉽습니다! 땡! 정답은 '{quiz['answer']}'이었어요.")
                wrong_answers.append({
                    "question": quiz['question'],
                    "correct_answer": quiz['answer'],
                    "my_answer": user_answer if user_answer else "대답 없음"
                })
        
        if exit_game:
            break

        # --- 5. 최종 결과 및 오답노트 발표 ---
        speak(f"자, 이렇게 모든 퀴즈가 끝났습니다! 최종 점수는 {len(quiz_list)}문제 중 {score}문제 정답!")
        
        if wrong_answers:
            speak("오늘 틀린 문제를 바탕으로 오답노트를 화면에 띄워드릴게요. 꼭 복습하세요!")
            
            odap_note_prompt = f"""
            다음은 사용자가 틀린 퀴즈 목록입니다: {wrong_answers}
            각 문제에 대해 핵심 내용을 한 줄로 요약해서 오답노트를 만들어주세요.
            친절한 말투로 "1번 문제:", "2번 문제:" 와 같이 서식을 지켜주세요.
            """

            odap_note_response = client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": odap_note_prompt}],
                temperature=0.3
            )
            final_note = odap_note_response.choices[0].message.content

            print("\n--- 📝 퀴즈 갈매기의 오답노트 ---")
            print(final_note)
            print("--------------------------------\n")
        else:
            speak("와, 모든 문제를 다 맞히셨네요! 마! 당신은 롯데의 진정한 팬입니다!")

        # --- 6. 게임 재시작 여부 확인
        speak("퀴즈를 처음부터 다시 풀어보시겠어요? '네' 또는 '아니요'로 대답해주세요.")
        play_again_answer = listen()

        if "네" in play_again_answer or "예" in play_again_answer:
            speak("좋습니다! 그럼 새로운 문제로 다시 시작할게요!")
            continue
        else:
            speak("알겠습니다! 다음에 또 만나요! 안녕~")
            break

    except Exception as e:
        print(f"프로그램 실행 중 오류 발생: {e}")
        break

AI가 새로운 퀴즈를 출제하는 중입니다...
🤖: 안녕하세요, 저는 롯데 자이언츠 퀴즈쇼 호스트, 퀴즈 갈매기입니다! 지금부터 총 3문제를 풀어볼 거예요.
🤖: 자, 1번째 문제입니다! 롯데 자이언츠의 창단년도는 언제인가요?
🎙️듣고 있어요. 말씀해주세요..
👤사용자: 1975년
🤖: 딩동댕! 정답입니다!
🤖: 자, 2번째 문제입니다! 롯데 자이언츠의 홈 구장 이름은 무엇일까요?
🎙️듣고 있어요. 말씀해주세요..
👤사용자: 사직 야구장
🤖: 딩동댕! 정답입니다!
🤖: 자, 3번째 문제입니다! 롯데 자이언츠의 팀 색상은 무엇인가요?
🎙️듣고 있어요. 말씀해주세요..
👤사용자: 군청색
🤖: 아~ 아쉽습니다! 땡! 정답은 '블루와 레드'이었어요.
🤖: 자, 이렇게 모든 퀴즈가 끝났습니다! 최종 점수는 3문제 중 2문제 정답!
🤖: 오늘 틀린 문제를 바탕으로 오답노트를 화면에 띄워드릴게요. 꼭 복습하세요!

--- 📝 퀴즈 갈매기의 오답노트 ---
1번 문제: 롯데 자이언츠의 팀 색상은 블루와 레드입니다, 군청색이 아닙니다.
--------------------------------

🤖: 퀴즈를 처음부터 다시 풀어보시겠어요? '네' 또는 '아니요'로 대답해주세요.
🎙️듣고 있어요. 말씀해주세요..
👤사용자: 아니요
🤖: 알겠습니다! 다음에 또 만나요! 안녕~
